In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import zipfile
from keras.layers import Dense, Reshape
import matplotlib.pyplot as plt
import math
from keras.losses import *
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape
from keras.layers import LSTM, Flatten, Input, Embedding, concatenate, Concatenate, average, maximum, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard


Using TensorFlow backend.


In [2]:
df = pd.read_pickle("data/dataframe.pd")
df

,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,...,id_37_T,id_38_F,id_38_T,DeviceType_desktop,DeviceType_mobile,DeviceInfo_MacOS,DeviceInfo_Trident/7.0,DeviceInfo_Windows,DeviceInfo_iOS Device,DeviceInfo_other
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.50,13926,362.555488,150.0,142.0,315.0,87.0,19.00000,...,0,0,0,0,0,0,0,0,0,0
2987001,0,86401,29.00,2755,404.000000,150.0,102.0,325.0,87.0,118.50218,...,0,0,0,0,0,0,0,0,0,0
2987002,0,86469,59.00,4663,490.000000,150.0,166.0,330.0,87.0,287.00000,...,0,0,0,0,0,0,0,0,0,0
2987003,0,86499,50.00,18132,567.000000,150.0,117.0,476.0,87.0,118.50218,...,0,0,0,0,0,0,0,0,0,0
2987004,0,86506,50.00,4497,514.000000,150.0,102.0,420.0,87.0,118.50218,...,1,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0,15811047,49.00,6550,362.555488,150.0,226.0,272.0,87.0,48.00000,...,0,0,0,0,0,0,0,0,0,0
3577536,0,15811049,39.50,10444,225.000000,150.0,224.0,204.0,87.0,118.50218,...,0,0,0,0,0,0,0,0,0,0
3577537,0,15811079,30.95,12037,595.000000,150.0,224.0,231.0,87.0,118.50218,...,0,0,0,0,0,0,0,0,0,0


In [3]:
percent_fraud = len(df.loc[df.isFraud == 0, :])/len(df)
percent_fraud

0.9650099908558268

In [ ]:
from sklearn.utils import class_weight
x_data = np.array(df.drop(["isFraud"], axis=1))
y_data= np.array(df['isFraud'])

scaler = MinMaxScaler(feature_range=(-1, 1))
x_data = scaler.fit_transform(x_data)



x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=.2, random_state=42, shuffle=False )

inputs = Input(shape=(312,))
# We stack a deep densely-connected network on top
x = Dense(400, activation='relu', use_bias=True)(inputs)
x = Dense(400,  activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
# And finally we add the main logistic regression layer
output = Dense(1 , name='ANN_output')(x)

adam = optimizers.Adam(lr=0.01)
model = Model(inputs=inputs, outputs = output)


weights = class_weight.compute_class_weight('balanced',
                                            np.unique(y_train),
                                            y_train)

model.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2 )#, class_weight= weights)
print()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 36s - loss: 21.7364 - acc: 0.9612
Epoch 2/50
 - 36s - loss: 0.0309 - acc: 0.9658
Epoch 3/50
 - 37s - loss: 0.0294 - acc: 0.9673
Epoch 4/50
 - 48s - loss: 0.0296 - acc: 0.9673
Epoch 5/50
 - 36s - loss: 0.0289 - acc: 0.9680
Epoch 6/50
 - 36s - loss: 0.0286 - acc: 0.9683
Epoch 7/50
 - 36s - loss: 0.0297 - acc: 0.9678
Epoch 8/50
 - 36s - loss: 0.0283 - acc: 0.9684
Epoch 9/50
 - 36s - loss: 0.0281 - acc: 0.9686
Epoch 10/50
 - 35s - loss: 0.0280 - acc: 0.9687
Epoch 11/50
 - 36s - loss: 0.0279 - acc: 0.9687
Epoch 12/50
 - 38s - loss: 0.0278 - acc: 0.9688
Epoch 13/50
 - 39s - loss: 0.0277 - acc: 0.9689
Epoch 14/50
 - 37s - loss: 0.0278 - acc: 0.9689
Epoch 15/50
 - 39s - loss: 0.0277 - acc: 0.9690
Epoch 16/50
 - 37s - loss: 0.0275 - acc: 0.9692
Epoch 17/50
 - 43s - loss: 0.0275 - acc: 0.9692
Epoch 18/50
 - 37s - loss: 0.0273 - acc: 0.9694
Epoch 19/50
 - 40s - los

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#trainScore = numpy.mean(numpy.abs((y_train - trainPredict))/y_train)

print('Train Score: %.2f MAPE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test, testPredict))
#testScore =  numpy.mean(numpy.abs((y_test - testPredict))/y_test)

print('Test Score: %.2f MAPE' % (testScore))
keras.metrics.binary_accuracy(y_true, y_pred)

